In [6]:
!pip install rasterio pyvista trimesh shapely

import rasterio
import numpy as np
import pyvista as pv
import trimesh

# Ruta al raster
ruta = '/content/gebco_2025.tif'

# --- Leer el GeoTIFF ---
with rasterio.open(ruta) as src:
    elev = src.read(1).astype(float)  # matriz de elevación
    transform = src.transform

# Crear coordenadas X, Y
rows, cols = elev.shape
x = np.arange(cols) * transform.a + transform.c
y = np.arange(rows) * transform.e + transform.f
X, Y = np.meshgrid(x, y)

# --- Crear grilla estructurada ---
grid = pv.StructuredGrid(X, Y, elev)

# --- Convertir StructuredGrid → PolyData (triangular) ---
surf = grid.extract_surface().triangulate()

# Extraer vértices y caras
vertices = surf.points
faces = surf.faces.reshape(-1, 4)[:, 1:]  # cada fila: (3, v0, v1, v2)

# Crear malla con Trimesh
mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

# --- Exportar a glTF (.glb) ---
mesh.export("gebco_2025.glb")
print("✅ Exportado a gebco_2025.glb")


✅ Exportado a gebco_2025.glb
